In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
test = pd.read_pickle("../../bld/out/data/test-kw_97_extended.pkl")
pred = pd.read_pickle("../../bld/out/analysis/kw_97_extended/predictions.pkl")

surrogate_model = 'neuralnet_n-20000_f_layers_88-88_n_epochs_200_n_batch_size_128_p_None'

test = test.reset_index(drop=True)
test = test.rename(columns={"qoi": "true"})

test["pred"] = pred[surrogate_model]

test = test.sort_values("delta")

mini, maxi = test["delta"].min(), test["delta"].max()
mini = np.floor(mini * 100) / 100
delta_grid = np.linspace(mini, maxi, 150)

test["delta_binned"] = pd.cut(test["delta"], delta_grid)

df = test.groupby("delta_binned").mean()[["delta", "true", "pred"]]

df = df.reset_index()
df["delta"] = df["delta_binned"].map(lambda interval: interval.left).astype(float)

df = df.drop("delta_binned", axis=1)
df = df.melt(id_vars="delta", var_name="type", value_name="qoi")

fig, ax = plt.subplots(figsize=(11, 6))
sns.lineplot(
    x="delta", y="qoi", hue="type", alpha=0.75, data=df, ax=ax, legend=None, linewidth=3
)
plt.legend(
    loc='lower left', labels=['True Model', 'Surrogate Model'], prop={"size": 12}, frameon=False
)
plt.xlabel("Delta", fontsize=12)
plt.ylabel("Quantity of Interest", fontsize=12)
fig.savefig("true-vs-surrogate.png", bbox_inches="tight")